# 🎙️ Deepfake Voice Detection System
### In-the-Wild Dataset — CNN + MFCC Pipeline

| Step | Description |
|------|-------------|
| 1 | Data Loading & MFCC Extraction |
| 2 | Data Preprocessing & Splitting |
| 3 | CNN Model Definition |
| 4 | Model Training |
| 5 | Model Testing & Metrics |
| 6 | Verification Conditions |
| 7 | Conditional Model Saving |
| 8 | Prediction Function |
| 9 | Final Testing (5 real + 5 fake) |

---
## 📦 Imports & Configuration

In [ ]:
# ── Install missing dependencies (run once) ──────────────────────────────────
# Uncomment the line below if librosa is not installed:
# !pip install librosa tqdm scikit-learn

import os
import random
import warnings
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, f1_score
)
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")
print("All imports successful!")

In [ ]:
# ── Configuration ─────────────────────────────────────────────────────────────

# Notebook lives inside: release_in_the_wild/
# Audio + meta.csv live in the nested: release_in_the_wild/release_in_the_wild/
_NB_DIR     = os.getcwd()                              # current working directory
DATASET_DIR = os.path.join(_NB_DIR, "release_in_the_wild")

# Fallback: if the nested folder doesn't exist, look in the current dir
if not os.path.isdir(DATASET_DIR):
    DATASET_DIR = _NB_DIR

META_CSV    = os.path.join(DATASET_DIR, "meta.csv")
MODEL_PATH  = os.path.join(_NB_DIR, "deepfake_voice_detector.pth")

# ── Hyper-parameters ──────────────────────────────────────────────────────────
N_MFCC        = 40          # number of MFCC coefficients
MAX_LEN       = 174         # fixed time-frame length (~4 s @ 22 050 Hz)
SAMPLE_RATE   = 22_050      # librosa default sample rate
EPOCHS        = 20
BATCH_SIZE    = 64
LEARNING_RATE = 1e-3
SEED          = 42

# ── Reproducibility ───────────────────────────────────────────────────────────
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Dataset dir : {DATASET_DIR}")
print(f"meta.csv    : {META_CSV}  →  exists={os.path.isfile(META_CSV)}")
print(f"Device      : {DEVICE}")

---
## Step 1 — Data Loading & MFCC Extraction

- Load each `.wav` with **librosa** at 22 050 Hz
- Extract **40 MFCC coefficients** per frame
- Pad (zeros) or trim to exactly **174 frames** → fixed shape `(40, 174)`
- Label mapping: `bona-fide` → **0 (Real)**, `spoof` → **1 (Fake)**

In [ ]:
def extract_mfcc(file_path: str, n_mfcc: int = N_MFCC, max_len: int = MAX_LEN):
    """
    Load a .wav file with librosa, compute MFCC, pad/trim to fixed length.
    Returns np.ndarray of shape (n_mfcc, max_len), or None on error.
    """
    try:
        y, sr = librosa.load(file_path, sr=SAMPLE_RATE, mono=True)
        mfcc  = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)  # (40, T)

        # Pad or trim along the time axis
        if mfcc.shape[1] < max_len:
            pad_width = max_len - mfcc.shape[1]
            mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode="constant")
        else:
            mfcc = mfcc[:, :max_len]

        return mfcc  # (40, 174)
    except Exception as e:
        print(f"[WARN] Could not load {file_path}: {e}")
        return None


def load_dataset():
    """
    Read meta.csv, extract MFCC for every audio file, assign numeric labels.
    Returns: X (N,40,174), y (N,), valid_files list
    """
    print("[STEP 1] Loading dataset …")
    df = pd.read_csv(META_CSV)

    label_map = {"bona-fide": 0, "spoof": 1}
    features, labels, valid_files = [], [], []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Extracting MFCCs"):
        audio_path = os.path.join(DATASET_DIR, row["file"])
        if not os.path.isfile(audio_path):
            continue

        mfcc = extract_mfcc(audio_path)
        if mfcc is None:
            continue

        features.append(mfcc)
        labels.append(label_map[row["label"]])
        valid_files.append(row["file"])

    X = np.array(features, dtype=np.float32)  # (N, 40, 174)
    y = np.array(labels,   dtype=np.int64)     # (N,)

    print(f"Loaded {len(X)} samples  (real={np.sum(y==0)}, fake={np.sum(y==1)})")
    return X, y, valid_files


# Run data loading (takes ~14 min on CPU for ~31 780 files)
X, y, valid_files = load_dataset()

---
## Step 2 — Data Preprocessing & Splitting

- Flatten + `StandardScaler` → reshape to `(N, 1, 40, 174)` (channel dim for CNN)
- Stratified split: **70% train / 15% val / 15% test**

In [ ]:
def preprocess(X: np.ndarray, y: np.ndarray):
    """
    Normalize with StandardScaler and split 70 / 15 / 15 (stratified).
    Returns: X_train, X_val, X_test, y_train, y_val, y_test, scaler
    """
    print("[STEP 2] Preprocessing …")
    N, H, W = X.shape

    # Normalize
    X_flat = X.reshape(N, -1)
    scaler  = StandardScaler()
    X_norm  = scaler.fit_transform(X_flat).reshape(N, 1, H, W)  # add channel dim

    # 70 / 30  →  then 30 split 50/50  =  15 / 15
    X_train, X_tmp, y_train, y_tmp = train_test_split(
        X_norm, y, test_size=0.30, random_state=SEED, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_tmp, y_tmp, test_size=0.50, random_state=SEED, stratify=y_tmp
    )

    print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")
    return X_train, X_val, X_test, y_train, y_val, y_test, scaler


def to_dataloader(X: np.ndarray, y: np.ndarray, batch_size: int, shuffle: bool):
    """Convert numpy arrays to a PyTorch DataLoader."""
    X_t = torch.tensor(X, dtype=torch.float32)
    y_t = torch.tensor(y, dtype=torch.long)
    return DataLoader(TensorDataset(X_t, y_t), batch_size=batch_size, shuffle=shuffle)


X_train, X_val, X_test, y_train, y_val, y_test, scaler = preprocess(X, y)

train_loader = to_dataloader(X_train, y_train, BATCH_SIZE, shuffle=True)
val_loader   = to_dataloader(X_val,   y_val,   BATCH_SIZE, shuffle=False)
test_loader  = to_dataloader(X_test,  y_test,  BATCH_SIZE, shuffle=False)

---
## Step 3 — CNN Model Definition

```
Input: (batch, 1, 40, 174)
  → Conv2d(1→32)+BN+ReLU+MaxPool2d  →  (32, 20, 87)
  → Conv2d(32→64)+BN+ReLU+MaxPool2d →  (64, 10, 43)
  → Conv2d(64→128)+BN+ReLU+MaxPool2d → (128, 5, 21)
  → Flatten → 13,440
  → FC(13440→256)+Dropout(0.4)
  → FC(256→64)+Dropout(0.4)
  → FC(64→2)   ← 2 output neurons
```

In [ ]:
class DeepfakeCNN(nn.Module):
    """
    CNN for deepfake voice detection.
    Input  : (batch, 1, 40, 174)  — 1-channel MFCC spectrogram
    Output : (batch, 2)           — logits for [real, fake]
    """

    def __init__(self):
        super(DeepfakeCNN, self).__init__()

        # Block 1: Conv → BN → ReLU → MaxPool
        self.conv1 = nn.Conv2d(1,   32, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)      # 40×174 → 20×87

        # Block 2
        self.conv2 = nn.Conv2d(32,  64, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)      # 20×87  → 10×43

        # Block 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)      # 10×43  → 5×21

        self.relu = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(0.4)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 5 * 21, 256)   # 13 440 → 256
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64,  2)               # 2 output neurons

    def forward(self, x):
        x = self.pool1(self.relu(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)               # flatten
        x = self.drop(self.relu(self.fc1(x)))
        x = self.drop(self.relu(self.fc2(x)))
        return self.fc3(x)                       # raw logits


print("[STEP 3] Building CNN model …")
model = DeepfakeCNN().to(DEVICE)
print(model)

---
## Step 4 — Model Training

- **Optimizer**: Adam (lr = 1e-3)
- **Loss**: CrossEntropyLoss
- **Epochs**: 20 (minimum 10 required)
- **Scheduler**: ReduceLROnPlateau (patience=3, factor=0.5)
- Prints train loss, train acc, val loss, val acc every epoch

In [ ]:
def train_model(model, train_loader, val_loader):
    """
    Train the CNN for EPOCHS using Adam + CrossEntropyLoss.
    Returns: final_val_loss (float), training_ok (bool)
    """
    print(f"[STEP 4] Training for {EPOCHS} epochs …\n")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=3, factor=0.5
    )

    training_ok    = True
    final_val_loss = float("nan")

    for epoch in range(1, EPOCHS + 1):
        # ── Training phase ────────────────────────────────────────────────────
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss    = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss    += loss.item() * X_batch.size(0)
            preds          = outputs.argmax(dim=1)
            train_correct += (preds == y_batch).sum().item()
            train_total   += X_batch.size(0)

        avg_train_loss = train_loss / train_total
        train_acc      = train_correct / train_total

        # ── Validation phase ──────────────────────────────────────────────────
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
                outputs     = model(X_batch)
                loss        = criterion(outputs, y_batch)
                val_loss   += loss.item() * X_batch.size(0)
                preds       = outputs.argmax(dim=1)
                val_correct += (preds == y_batch).sum().item()
                val_total   += X_batch.size(0)

        avg_val_loss   = val_loss / val_total
        val_acc        = val_correct / val_total
        final_val_loss = avg_val_loss

        scheduler.step(avg_val_loss)

        # Guard against NaN loss
        if np.isnan(avg_train_loss) or np.isnan(avg_val_loss):
            print(f"[ERROR] NaN loss at epoch {epoch}. Stopping.")
            training_ok = False
            break

        print(
            f"Epoch [{epoch:>2}/{EPOCHS}]  "
            f"Train Loss: {avg_train_loss:.4f}  Train Acc: {train_acc:.4f}  |  "
            f"Val Loss: {avg_val_loss:.4f}  Val Acc: {val_acc:.4f}"
        )

    return final_val_loss, training_ok


final_val_loss, training_ok = train_model(model, train_loader, val_loader)

---
## Step 5 — Model Testing & Metrics

Evaluate on the held-out **test set** (15% of total data) and report:
- ✅ Test Accuracy
- ✅ Confusion Matrix
- ✅ Precision
- ✅ Recall
- ✅ F1 Score

In [ ]:
def evaluate_model(model, test_loader):
    """
    Evaluate on the test set.
    Returns test_accuracy (float).
    """
    print("[STEP 5] Evaluating on test set …")
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(DEVICE)
            preds   = model(X_batch).argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(y_batch.numpy())

    all_preds  = np.array(all_preds)
    all_labels = np.array(all_labels)

    accuracy  = np.mean(all_preds == all_labels)
    cm        = confusion_matrix(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall    = recall_score(all_labels, all_preds, zero_division=0)
    f1        = f1_score(all_labels, all_preds, zero_division=0)

    print(f"\n  Test Accuracy  : {accuracy * 100:.2f}%")
    print(f"  Confusion Matrix (rows=actual, cols=predicted):")
    print(f"  {cm}")
    print(f"  Precision      : {precision:.4f}")
    print(f"  Recall         : {recall:.4f}")
    print(f"  F1 Score       : {f1:.4f}")

    return accuracy


test_accuracy = evaluate_model(model, test_loader)

---
## Steps 6 & 7 — Verification + Conditional Model Saving

Model is saved **only if ALL four conditions pass**:

| Gate | Condition |
|------|-----------|
| ✔ | Test accuracy ≥ 85% |
| ✔ | Training completed without errors |
| ✔ | Validation loss is not NaN |
| ✔ | ≥ 5 out of 20 random samples predicted correctly |

In [ ]:
def verify_and_save(model, scaler, test_accuracy, final_val_loss,
                    training_ok, X_test, y_test):
    """
    Run all 4 verification gates. Save deepfake_voice_detector.pth only if all pass.
    Returns: all_passed (bool)
    """
    print("[STEP 6] Model Verification …\n")

    # Gate 1 – accuracy
    cond_acc = test_accuracy >= 0.85
    print(f"  [{'PASS' if cond_acc else 'FAIL'}] Test accuracy >= 85%  "
          f"({test_accuracy*100:.2f}%)")

    # Gate 2 – no training error
    cond_train = training_ok
    print(f"  [{'PASS' if cond_train else 'FAIL'}] Training completed without errors")

    # Gate 3 – val loss not NaN
    cond_loss = not np.isnan(final_val_loss)
    print(f"  [{'PASS' if cond_loss else 'FAIL'}] Validation loss is not NaN  "
          f"({final_val_loss:.4f})")

    # Gate 4 – random sample correctness
    model.eval()
    random_indices = random.sample(range(len(X_test)), min(20, len(X_test)))
    correct_count  = 0
    with torch.no_grad():
        for idx in random_indices:
            x_s  = torch.tensor(X_test[idx:idx+1], dtype=torch.float32).to(DEVICE)
            pred = model(x_s).argmax(dim=1).item()
            if pred == y_test[idx]:
                correct_count += 1
    cond_samples = correct_count >= 5
    print(f"  [{'PASS' if cond_samples else 'FAIL'}] >= 5/20 random samples correct  "
          f"({correct_count}/20 correct)")

    all_passed = cond_acc and cond_train and cond_loss and cond_samples

    print()
    if all_passed:
        print("[STEP 7] All conditions passed!")
        torch.save(
            {
                "model_state_dict": model.state_dict(),
                "scaler_mean" :     scaler.mean_,
                "scaler_scale":     scaler.scale_,
                "n_mfcc"      :     N_MFCC,
                "max_len"     :     MAX_LEN,
                "sample_rate" :     SAMPLE_RATE,
            },
            MODEL_PATH,
        )
        print("Model verified and saved successfully.")
    else:
        print("Model failed verification. Not saving.")

    return all_passed


all_passed = verify_and_save(
    model, scaler, test_accuracy, final_val_loss,
    training_ok, X_test, y_test
)

---
## Step 8 — Prediction Function

`predict_audio(file_path)` — loads any `.wav`, extracts MFCC, runs inference, and prints:
- `"Real Voice"` or `"Fake Voice"`

The model is kept in memory after training; if called standalone it lazy-loads from `deepfake_voice_detector.pth`.

In [ ]:
# Module-level references for lazy loading
_MODEL  = None
_SCALER = None


def _load_model_for_inference():
    """Load model + scaler from disk (used when running predict_audio standalone)."""
    checkpoint    = torch.load(MODEL_PATH, map_location=DEVICE)
    scaler        = StandardScaler()
    scaler.mean_  = checkpoint["scaler_mean"]
    scaler.scale_ = checkpoint["scaler_scale"]
    m = DeepfakeCNN().to(DEVICE)
    m.load_state_dict(checkpoint["model_state_dict"])
    m.eval()
    return m, scaler


def predict_audio(file_path: str):
    """
    Predict whether an audio file is a real or deepfake voice.

    Args:
        file_path : str — path to a .wav file

    Prints:
        "Real Voice"  or  "Fake Voice"

    Returns:
        str — "Real Voice" or "Fake Voice"
    """
    global _MODEL, _SCALER

    # Lazy-load model from disk if needed
    if _MODEL is None or _SCALER is None:
        if not os.path.isfile(MODEL_PATH):
            raise FileNotFoundError(
                f"No saved model at {MODEL_PATH}. "
                "Run the full training pipeline first."
            )
        _MODEL, _SCALER = _load_model_for_inference()

    mfcc = extract_mfcc(file_path)
    if mfcc is None:
        print(f"[ERROR] Could not process {file_path}")
        return None

    # Normalize with the training scaler
    flat   = mfcc.reshape(1, -1)
    norm   = _SCALER.transform(flat).reshape(1, 1, N_MFCC, MAX_LEN)
    tensor = torch.tensor(norm, dtype=torch.float32).to(DEVICE)

    _MODEL.eval()
    with torch.no_grad():
        pred = _MODEL(tensor).argmax(dim=1).item()

    label = "Fake Voice" if pred == 1 else "Real Voice"
    print(f"  [{os.path.basename(file_path)}]  →  {label}")
    return label


# If training passed, wire the in-memory model so predict_audio doesn't reload from disk
if all_passed:
    _MODEL  = model
    _SCALER = scaler

print("predict_audio() is ready.")

---
## Step 9 — Final Testing (5 Real + 5 Fake)

Run `predict_audio()` on randomly selected samples to confirm the model generalises.

In [ ]:
def final_test(valid_files, labels):
    """Test predict_audio on 5 real and 5 fake audio files."""
    print("[STEP 9] Final Testing — 5 real + 5 fake files\n")

    real_files = [f for f, l in zip(valid_files, labels) if l == 0]
    fake_files = [f for f, l in zip(valid_files, labels) if l == 1]

    sampled_real = random.sample(real_files, min(5, len(real_files)))
    sampled_fake = random.sample(fake_files, min(5, len(fake_files)))

    print(">> Real voice samples:")
    for fname in sampled_real:
        predict_audio(os.path.join(DATASET_DIR, fname))

    print("\n>> Fake voice samples:")
    for fname in sampled_fake:
        predict_audio(os.path.join(DATASET_DIR, fname))


if all_passed:
    final_test(valid_files, y)
else:
    print("[INFO] Skipping final test — model did not pass verification.")

---
## 🔍 Standalone Prediction (Optional)

After training is complete, you can call `predict_audio()` on any `.wav` file:

In [ ]:
# Example: predict on any file path
# predict_audio(r"C:\path\to\your\audio.wav")

# Or load from the dataset directory:
# predict_audio(os.path.join(DATASET_DIR, "0.wav"))

print("Uncomment a line above and provide a .wav path to test inference.")